In [1]:
#importing all the libraries that are required for the project
#we will be working with integrating pyspark with hdfs to read the files.
#then putting the output data in MySql.
#and finally visualising the data using grafana

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,IntegerType, StructType,StringType
import functools
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col

In [3]:
spark = SparkSession.builder \
        .config("spark.jars","/home/savvy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar") \
        .appName("reading2") \
        .getOrCreate()

23/04/04 00:24:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 00:24:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
#print(spark.sparkContext._jsc.sc().listJars())

In [4]:
print(spark.sparkContext)

<SparkContext master=local[*] appName=reading2>


In [6]:
readData = spark.read.csv("hdfs://localhost:9000/savvy/", header='true')
readData.show()

+----------+-----------+---------+---------+------+
|      Date|    Country|Confirmed|Recovered|Deaths|
+----------+-----------+---------+---------+------+
|2020-01-22|Afghanistan|        0|        0|     0|
|2020-01-23|Afghanistan|        0|        0|     0|
|2020-01-24|Afghanistan|        0|        0|     0|
|2020-01-25|Afghanistan|        0|        0|     0|
|2020-01-26|Afghanistan|        0|        0|     0|
|2020-01-27|Afghanistan|        0|        0|     0|
|2020-01-28|Afghanistan|        0|        0|     0|
|2020-01-29|Afghanistan|        0|        0|     0|
|2020-01-30|Afghanistan|        0|        0|     0|
|2020-01-31|Afghanistan|        0|        0|     0|
|2020-02-01|Afghanistan|        0|        0|     0|
|2020-02-02|Afghanistan|        0|        0|     0|
|2020-02-03|Afghanistan|        0|        0|     0|
|2020-02-04|Afghanistan|        0|        0|     0|
|2020-02-05|Afghanistan|        0|        0|     0|
|2020-02-06|Afghanistan|        0|        0|     0|
|2020-02-07|

In [7]:
readData.groupBy("Country").count().show()

+-----------+-----+
|    Country|count|
+-----------+-----+
|       Chad|  476|
|   Paraguay|  476|
|     Russia|  476|
|      Yemen|  476|
|    Senegal|  476|
| Cabo Verde|  476|
|     Sweden|  476|
|     Guyana|  476|
|      Burma|  476|
|    Eritrea|  476|
|Philippines|  476|
|   Djibouti|  476|
|   Malaysia|  476|
|  Singapore|  476|
|       Fiji|  476|
|     Turkey|  476|
|     Malawi|  476|
|       Iraq|  476|
|    Germany|  476|
|    Comoros|  476|
+-----------+-----+
only showing top 20 rows



In [8]:
readData.groupBy("Country").sum("Confirmed").show()

AnalysisException: "Confirmed" is not a numeric column. Aggregation function can only be applied on a numeric column.

In [9]:
readData.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Deaths: string (nullable = true)



In [10]:
df =[StructField('Date',IntegerType(),True),
       StructField('Country',StringType(),True),
       StructField('Confirmed',IntegerType(),True),
       StructField('Recovered',IntegerType(),True),
       StructField('Deaths',IntegerType(),True),
       ]

finalStruct=StructType(fields=df)
readData2=spark.read.csv('hdfs://localhost:9000/savvy/',schema=finalStruct)

In [11]:
readData2.printSchema()

root
 |-- Date: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Deaths: integer (nullable = true)



In [12]:
df1 = readData2.groupBy("Country").sum("Confirmed")
df1.show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|        795278|
|   Paraguay|      32348042|
|     Russia|     841949638|
|      Yemen|        842988|
|    Senegal|       7155286|
| Cabo Verde|       3421969|
|     Sweden|     123238302|
|     Guyana|       1784186|
|      Burma|      25139944|
|    Eritrea|        452505|
|Philippines|     143412277|
|   Djibouti|       2180594|
|   Malaysia|      44562079|
|  Singapore|      20365406|
|       Fiji|         16704|
|     Turkey|     526056925|
|     Malawi|       4618570|
|       Iraq|     166448684|
|    Germany|     458729889|
|    Comoros|        510069|
+-----------+--------------+
only showing top 20 rows



In [13]:
df2 = readData2.groupBy("Country").sum("Recovered")
df2.show()

+-----------+--------------+
|    Country|sum(Recovered)|
+-----------+--------------+
|       Chad|        686509|
|   Paraguay|      25368413|
|     Russia|     708948906|
|      Yemen|        449502|
|    Senegal|       6198659|
| Cabo Verde|       3106429|
|     Sweden|             0|
|     Guyana|       1502459|
|      Burma|      21680411|
|    Eritrea|        374647|
|Philippines|     123141517|
|   Djibouti|       2017792|
|   Malaysia|      39433161|
|  Singapore|      19033712|
|       Fiji|         13850|
|     Turkey|     474413925|
|     Malawi|       3414729|
|       Iraq|     145843021|
|    Germany|     392664956|
|    Comoros|        454618|
+-----------+--------------+
only showing top 20 rows



In [14]:
df3 = readData2.groupBy("Country").sum("Deaths")
df3.show()

+-----------+-----------+
|    Country|sum(Deaths)|
+-----------+-----------+
|       Chad|      38341|
|   Paraguay|     673403|
|     Russia|   16283420|
|      Yemen|     214212|
|    Senegal|     167476|
| Cabo Verde|      33154|
|     Sweden|    3078055|
|     Guyana|      44787|
|      Burma|     559260|
|    Eritrea|       1041|
|Philippines|    2735101|
|   Djibouti|      23073|
|   Malaysia|     191494|
|  Singapore|      10665|
|       Fiji|        549|
|     Turkey|    6019058|
|     Malawi|     146534|
|       Iraq|    3442246|
|    Germany|   11797190|
|    Comoros|      16892|
+-----------+-----------+
only showing top 20 rows



In [15]:
main_df = df1.union(df2).union(df3)
main_df.show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|        795278|
|   Paraguay|      32348042|
|     Russia|     841949638|
|      Yemen|        842988|
|    Senegal|       7155286|
| Cabo Verde|       3421969|
|     Sweden|     123238302|
|     Guyana|       1784186|
|      Burma|      25139944|
|    Eritrea|        452505|
|Philippines|     143412277|
|   Djibouti|       2180594|
|   Malaysia|      44562079|
|  Singapore|      20365406|
|       Fiji|         16704|
|     Turkey|     526056925|
|     Malawi|       4618570|
|       Iraq|     166448684|
|    Germany|     458729889|
|    Comoros|        510069|
+-----------+--------------+
only showing top 20 rows



In [16]:
#assigning ids to all the dataframes
conf_df = df1.withColumn("id",monotonically_increasing_id())
conf_df.show()
recov_df = df2.withColumn("id", monotonically_increasing_id())
recov_df.show()
dea_df = df3.withColumn("id",monotonically_increasing_id())
dea_df.show()

+-----------+--------------+---+
|    Country|sum(Confirmed)| id|
+-----------+--------------+---+
|       Chad|        795278|  0|
|   Paraguay|      32348042|  1|
|     Russia|     841949638|  2|
|      Yemen|        842988|  3|
|    Senegal|       7155286|  4|
| Cabo Verde|       3421969|  5|
|     Sweden|     123238302|  6|
|     Guyana|       1784186|  7|
|      Burma|      25139944|  8|
|    Eritrea|        452505|  9|
|Philippines|     143412277| 10|
|   Djibouti|       2180594| 11|
|   Malaysia|      44562079| 12|
|  Singapore|      20365406| 13|
|       Fiji|         16704| 14|
|     Turkey|     526056925| 15|
|     Malawi|       4618570| 16|
|       Iraq|     166448684| 17|
|    Germany|     458729889| 18|
|    Comoros|        510069| 19|
+-----------+--------------+---+
only showing top 20 rows

+-----------+--------------+---+
|    Country|sum(Recovered)| id|
+-----------+--------------+---+
|       Chad|        686509|  0|
|   Paraguay|      25368413|  1|
|     Russia|    

In [17]:
#joining conf_df and recov_df with the key "id" using .join method
conf_df.join(recov_df,conf_df["id"] == recov_df["id"]).show()

#drop duplicate column from recov_df
#conf_df.join(recov_df,["id","Country"]).show()

+-----------+--------------+---+-----------+--------------+---+
|    Country|sum(Confirmed)| id|    Country|sum(Recovered)| id|
+-----------+--------------+---+-----------+--------------+---+
|       Chad|        795278|  0|       Chad|        686509|  0|
|   Paraguay|      32348042|  1|   Paraguay|      25368413|  1|
|     Russia|     841949638|  2|     Russia|     708948906|  2|
|      Yemen|        842988|  3|      Yemen|        449502|  3|
|    Senegal|       7155286|  4|    Senegal|       6198659|  4|
| Cabo Verde|       3421969|  5| Cabo Verde|       3106429|  5|
|     Sweden|     123238302|  6|     Sweden|             0|  6|
|     Guyana|       1784186|  7|     Guyana|       1502459|  7|
|      Burma|      25139944|  8|      Burma|      21680411|  8|
|    Eritrea|        452505|  9|    Eritrea|        374647|  9|
|Philippines|     143412277| 10|Philippines|     123141517| 10|
|   Djibouti|       2180594| 11|   Djibouti|       2017792| 11|
|   Malaysia|      44562079| 12|   Malay

In [18]:
#joining joined data of conf_df and recov_df with dea_df
conf_df.join(recov_df,["id"]) \
     .join(dea_df,conf_df["id"] == dea_df["id"])\
     .show()

+---+-----------+--------------+-----------+--------------+-----------+-----------+---+
| id|    Country|sum(Confirmed)|    Country|sum(Recovered)|    Country|sum(Deaths)| id|
+---+-----------+--------------+-----------+--------------+-----------+-----------+---+
|  0|       Chad|        795278|       Chad|        686509|       Chad|      38341|  0|
|  1|   Paraguay|      32348042|   Paraguay|      25368413|   Paraguay|     673403|  1|
|  2|     Russia|     841949638|     Russia|     708948906|     Russia|   16283420|  2|
|  3|      Yemen|        842988|      Yemen|        449502|      Yemen|     214212|  3|
|  4|    Senegal|       7155286|    Senegal|       6198659|    Senegal|     167476|  4|
|  5| Cabo Verde|       3421969| Cabo Verde|       3106429| Cabo Verde|      33154|  5|
|  6|     Sweden|     123238302|     Sweden|             0|     Sweden|    3078055|  6|
|  7|     Guyana|       1784186|     Guyana|       1502459|     Guyana|      44787|  7|
|  8|      Burma|      25139944|

In [19]:
#removing duplicated columns of id and country
dataDF = conf_df.join(recov_df,["id","Country"]).join(dea_df,["id","Country"])
dataDF.show()

dataDF.printSchema()

+---+-----------+--------------+--------------+-----------+
| id|    Country|sum(Confirmed)|sum(Recovered)|sum(Deaths)|
+---+-----------+--------------+--------------+-----------+
|  0|       Chad|        795278|        686509|      38341|
|  1|   Paraguay|      32348042|      25368413|     673403|
|  2|     Russia|     841949638|     708948906|   16283420|
|  3|      Yemen|        842988|        449502|     214212|
|  4|    Senegal|       7155286|       6198659|     167476|
|  5| Cabo Verde|       3421969|       3106429|      33154|
|  6|     Sweden|     123238302|             0|    3078055|
|  7|     Guyana|       1784186|       1502459|      44787|
|  8|      Burma|      25139944|      21680411|     559260|
|  9|    Eritrea|        452505|        374647|       1041|
| 10|Philippines|     143412277|     123141517|    2735101|
| 11|   Djibouti|       2180594|       2017792|      23073|
| 12|   Malaysia|      44562079|      39433161|     191494|
| 13|  Singapore|      20365406|      19

In [20]:
data = dataDF.select(col("id"),col("Country"),col("sum(Confirmed)").alias("sum_Confirmed"),
                 col("sum(Recovered)").alias("sum_Recovered"),
                 col("sum(Deaths)").alias("sum_Deaths"))
data.show()
data.printSchema()

+---+-----------+-------------+-------------+----------+
| id|    Country|sum_Confirmed|sum_Recovered|sum_Deaths|
+---+-----------+-------------+-------------+----------+
|  0|       Chad|       795278|       686509|     38341|
|  1|   Paraguay|     32348042|     25368413|    673403|
|  2|     Russia|    841949638|    708948906|  16283420|
|  3|      Yemen|       842988|       449502|    214212|
|  4|    Senegal|      7155286|      6198659|    167476|
|  5| Cabo Verde|      3421969|      3106429|     33154|
|  6|     Sweden|    123238302|            0|   3078055|
|  7|     Guyana|      1784186|      1502459|     44787|
|  8|      Burma|     25139944|     21680411|    559260|
|  9|    Eritrea|       452505|       374647|      1041|
| 10|Philippines|    143412277|    123141517|   2735101|
| 11|   Djibouti|      2180594|      2017792|     23073|
| 12|   Malaysia|     44562079|     39433161|    191494|
| 13|  Singapore|     20365406|     19033712|     10665|
| 14|       Fiji|        16704|

In [21]:
#Inserting the final dataframe data into MySQL table 'result'

In [22]:
url = "jdbc:mysql://localhost/output_table"
properties = { \
        "driver":"com.mysql.cj.jdbc.Driver", \
        "user":"savvy", \
        "password":"savvy@123" \
        }

In [29]:
data.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", url) \
  .option("dbtable", "result") \
  .option("user", "savvy") \
  .option("password", "savvy@123") \
  .mode("Append").save()

In [ ]:
#connect to grafana
#in localhost:3000